# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with mongoimport mongoimport --db uk_food --collection establishments --file "Resources/establishments.json" --jsonArray


In [83]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [85]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [87]:
# confirm that our new database was created
databases = mongo.list_database_names()
print("Databases available:", databases)

Databases available: ['admin', 'classDB', 'config', 'fruits_db', 'local', 'uk_food']


In [89]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [91]:
# review the collections in our new database
collections = db.list_collection_names()
print("Collections in the 'uk_food' database:", collections)

Collections in the 'uk_food' database: ['establishments']


In [93]:
# review a document in the establishments collection
collection = db['establishments']
document = collection.find_one()
pprint(document)

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6762eee183cbdbce994b2306'),
 'geocode': {'latitude': 51.086058, 'longitude': 1.196408},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extrac

In [95]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [99]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}


In [101]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)

In [103]:
# Check that the new restaurant was inserted
added_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
print(added_restaurant)

{'_id': ObjectId('6763066ddbc62490b94deea9'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'Greenwich', 'PostCode': 'SE10 9HY', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityCode': '999', 'LocalAuthorityWebSite': 'http://www.greenwich.gov.uk/', 'SchemeType': 'FHRS', 'geocode': {'longitude': 0.0, 'latitude': 0.0}, 'NewRatingPending': True}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [106]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},  
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}  
)
print(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [109]:
# Update the new restaurant with the correct BusinessTypeID
update_result = establishments.update_one(
    {"BusinessName": "Penang Flavours"}, 
    {"$set": {"BusinessTypeID": 1}}     
)

In [110]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(updated_restaurant)

{'AddressLine1': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'LocalAuthorityCode': '999',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.greenwich.gov.uk/',
 'NewRatingPending': True,
 'PostCode': 'SE10 9HY',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6763066ddbc62490b94deea9'),
 'geocode': {'latitude': 0.0, 'longitude': 0.0}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [114]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(dover_count)

0


In [116]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})

In [118]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})

In [120]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()
pprint(remaining_document)

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6762eee183cbdbce994b2306'),
 'geocode': {'latitude': 51.086058, 'longitude': 1.196408},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extrac

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [124]:
# Change the data type from String to Decimal for longitude and latitude
update_result = establishments.update_many(
    {},  # Match all documents in the collection
    [
        {
            "$set": {
                "geocode.latitude": {"$toDouble": "$geocode.latitude"},
                "geocode.longitude": {"$toDouble": "$geocode.longitude"}
            }
        }
    ]
)

Use `update_many` to convert `RatingValue` to integer numbers.

In [127]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [129]:
# Change the data type from String to Integer for RatingValue
update_numeric_to_int = establishments.update_many(
    {"RatingValue": {"$type": "string"}},   
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]  
)

In [131]:
# Check that the coordinates and rating value are now numbers
coordinates_sample = establishments.find(
    {},
    {"geocode.latitude": 1, "geocode.longitude": 1, "RatingValue": 1, "_id": 0}  # Include relevant fields only
).limit(10)

print("Sample documents to verify coordinates and RatingValue:")
from pprint import pprint
pprint(list(coordinates_sample))

Sample documents to verify coordinates and RatingValue:
[{'RatingValue': 4, 'geocode': {'latitude': 51.086058, 'longitude': 1.196408}},
 {'RatingValue': 5, 'geocode': {'latitude': 51.085797, 'longitude': 1.194762}},
 {'RatingValue': 5, 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}},
 {'RatingValue': 5, 'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'RatingValue': 5, 'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'RatingValue': 4, 'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'RatingValue': 5, 'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'RatingValue': 5,
  'geocode': {'latitude': 51.0783519967076, 'longitude': 1.18590330311705}},
 {'RatingValue': 5,
  'geocode': {'latitude': 51.0783519967076, 'longitude': 1.18590330311705}},
 {'RatingValue': 4,
  'geocode': {'latitude': 51.0783519967076, 'longitude': 1.18590330311705}}]
